<a href="https://colab.research.google.com/github/ahmadulfi/dantion/blob/main/ML-Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

logo_dantion.svg